In [66]:
!pip install opencv-python

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [68]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\Cars0.xml,226,419,125,173
1,./images\Cars1.xml,134,262,128,160
2,./images\Cars10.xml,140,303,5,148
3,./images\Cars100.xml,175,214,114,131
4,./images\Cars101.xml,167,240,202,220


In [69]:
import xml.etree.ElementTree as xet

In [70]:
filename = df['filepath'][0]
filename

'./images\\Cars0.xml'

In [71]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images', filename_image)
    return filepath_image

In [72]:
getFilename(filename)

'./images\\Cars0.png'

In [75]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['./images\\Cars0.png',
 './images\\Cars1.png',
 './images\\Cars10.png',
 './images\\Cars100.png',
 './images\\Cars101.png',
 './images\\Cars102.png',
 './images\\Cars103.png',
 './images\\Cars104.png',
 './images\\Cars105.png',
 './images\\Cars106.png',
 './images\\Cars107.png',
 './images\\Cars108.png',
 './images\\Cars109.png',
 './images\\Cars11.png',
 './images\\Cars110.png',
 './images\\Cars111.png',
 './images\\Cars112.png',
 './images\\Cars113.png',
 './images\\Cars114.png',
 './images\\Cars115.png',
 './images\\Cars116.png',
 './images\\Cars117.png',
 './images\\Cars118.png',
 './images\\Cars119.png',
 './images\\Cars12.png',
 './images\\Cars120.png',
 './images\\Cars121.png',
 './images\\Cars122.png',
 './images\\Cars123.png',
 './images\\Cars124.png',
 './images\\Cars125.png',
 './images\\Cars126.png',
 './images\\Cars127.png',
 './images\\Cars128.png',
 './images\\Cars129.png',
 './images\\Cars13.png',
 './images\\Cars130.png',
 './images\\Cars131.png',
 './images\\Cars132.

In [76]:
file_path = image_path[1]
file_path

'./images\\Cars1.png'

In [77]:
img = cv2.imread(file_path)

cv2.namedWindow('example', cv2.WINDOW_NORMAL)
cv2.imshow('example', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [79]:
#134	262	128	160
cv2.rectangle(img,(134,128),(262,160),(0,255,0),3)

cv2.namedWindow('example', cv2.WINDOW_NORMAL)
cv2.imshow('example', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [80]:
!pip install tensorflow

In [81]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [82]:
labels = df.iloc[:,1:].values

In [83]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape

    #Prepprocesing
    load_image = load_img(image, target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0

    #Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin, nxmax = xmin/w, xmax/w
    nymin, nymax = ymin/h, ymax/h
    label_norm = (nxmin, nxmax, nymin, nymax)
    
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [84]:
X = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)

In [85]:
X.shape, y.shape

((433, 224, 224, 3), (433, 4))

In [86]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape


((346, 224, 224, 3), (87, 224, 224, 3), (346, 4), (87, 4))

In [87]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf


In [88]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                    input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False

headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation="relu")(headmodel)
headmodel = Dense(250, activation="relu")(headmodel)
headmodel = Dense(4, activation="sigmoid")(headmodel)


In [89]:
model = Model(inputs=inception_resnet.input, outputs=headmodel)

In [90]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_203 (Conv2D)            (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_203 (Batch  (None, 111, 111, 32  96         ['conv2d_203[0][0]']             
 Normalization)                 )                                                           

 batch_normalization_208 (Batch  (None, 25, 25, 96)  288         ['conv2d_208[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_210 (Batch  (None, 25, 25, 64)  192         ['conv2d_210[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_213 (Batch  (None, 25, 25, 96)  288         ['conv2d_213[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_214 (Batch  (None, 25, 25, 64)  192         ['conv2d_214[0][0]']             
 Normaliza

 activation_224 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_224[0][0]']
                                                                                                  
 conv2d_222 (Conv2D)            (None, 25, 25, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_225 (Conv2D)            (None, 25, 25, 48)   13824       ['activation_224[0][0]']         
                                                                                                  
 batch_normalization_222 (Batch  (None, 25, 25, 32)  96          ['conv2d_222[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_225 (Batch  (None, 25, 25, 48)  144         ['conv2d_225[0][0]']             
 Normaliza

                                                                                                  
 activation_227 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_227[0][0]']
                                                                                                  
 activation_229 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_229[0][0]']
                                                                                                  
 activation_232 (Activation)    (None, 25, 25, 64)   0           ['batch_normalization_232[0][0]']
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 25, 25, 128)  0           ['activation_227[0][0]',         
                                                                  'activation_229[0][0]',         
                                                                  'activation_232[0][0]']         
          

 Normalization)                                                                                   
                                                                                                  
 batch_normalization_243 (Batch  (None, 25, 25, 48)  144         ['conv2d_243[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_240 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_240[0][0]']
                                                                                                  
 activation_243 (Activation)    (None, 25, 25, 48)   0           ['batch_normalization_243[0][0]']
                                                                                                  
 conv2d_239 (Conv2D)            (None, 25, 25, 32)   10240       ['block35_4_ac[0][0]']           
          

...

                                                                                                  
 activation_372 (Activation)    (None, 5, 5, 224)    0           ['batch_normalization_372[0][0]']
                                                                                                  
 conv2d_370 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_373 (Conv2D)            (None, 5, 5, 256)    172032      ['activation_372[0][0]']         
                                                                                                  
 batch_normalization_370 (Batch  (None, 5, 5, 192)   576         ['conv2d_370[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_nor

 batch_normalization_378 (Batch  (None, 5, 5, 192)   576         ['conv2d_378[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_381 (Batch  (None, 5, 5, 256)   768         ['conv2d_381[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_378 (Activation)    (None, 5, 5, 192)    0           ['batch_normalization_378[0][0]']
                                                                                                  
 activation_381 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_381[0][0]']
                                                                                                  
 block8_4_

                                                                                                  
 activation_389 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_389[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 5, 5, 448)    0           ['activation_386[0][0]',         
                                                                  'activation_389[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 5, 5, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 block8_6 (Lambda)              (None, 5, 5, 2080)   0           ['block8_5_ac[0][0]',            
                                                                  'block8_6_conv[0][0]']          
          

                                                                                                  
 block8_8 (Lambda)              (None, 5, 5, 2080)   0           ['block8_7_ac[0][0]',            
                                                                  'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 5, 5, 2080)   0           ['block8_8[0][0]']               
                                                                                                  
 conv2d_399 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_normalization_399 (Batch  (None, 5, 5, 192)   576         ['conv2d_399[0][0]']             
 Normalization)                                                                                   
          

 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 5, 5, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
 flatten_1 (Flatten)            (None, 38400)        0           ['conv_7b_ac[0][0]']             
                                                                                                  
 dense_3 (Dense)                (None, 500)          19200500    ['flatten_1[0][0]']              
                                                                                                  
 dense_4 (Dense)                (None, 250)          125250      ['dense_3[0][0]']                
                                                                                                  
 dense_5 (

In [91]:
from tensorflow.keras.callbacks import TensorBoard

In [92]:
tfb = TensorBoard('object_detection')

In [93]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=100,
                   validation_data=(x_test, y_test), callbacks=[tfb])

Epoch 1/100
35/35 [==============================] - 61s 2s/step - loss: 0.1256 - val_loss: 0.1287
Epoch 2/100
35/35 [==============================] - 53s 2s/step - loss: 0.1338 - val_loss: 0.1317
Epoch 3/100
35/35 [==============================] - 54s 2s/step - loss: 0.1268 - val_loss: 0.1256
Epoch 4/100
35/35 [==============================] - 59s 2s/step - loss: 0.1095 - val_loss: 0.0976
Epoch 5/100
35/35 [==============================] - 55s 2s/step - loss: 0.0934 - val_loss: 0.0944
Epoch 6/100
35/35 [==============================] - 54s 2s/step - loss: 0.0841 - val_loss: 0.0897
Epoch 7/100
35/35 [==============================] - 54s 2s/step - loss: 0.0779 - val_loss: 0.0850
Epoch 8/100
35/35 [==============================] - 57s 2s/step - loss: 0.0757 - val_loss: 0.0873
Epoch 9/100
35/35 [==============================] - 54s 2s/step - loss: 0.0757 - val_loss: 0.0846
Epoch 10/100
35/35 [==============================] - 57s 2s/step - loss: 0.0750 - val_loss: 0.0821
Epoch 11/

35/35 [==============================] - 54s 2s/step - loss: 5.3958e-04 - val_loss: 0.0157
Epoch 82/100
35/35 [==============================] - 55s 2s/step - loss: 4.9756e-04 - val_loss: 0.0148
Epoch 83/100
35/35 [==============================] - 55s 2s/step - loss: 5.3840e-04 - val_loss: 0.0153
Epoch 84/100
35/35 [==============================] - 55s 2s/step - loss: 5.3980e-04 - val_loss: 0.0155
Epoch 85/100
35/35 [==============================] - 54s 2s/step - loss: 4.8096e-04 - val_loss: 0.0153
Epoch 86/100
35/35 [==============================] - 55s 2s/step - loss: 5.3072e-04 - val_loss: 0.0154
Epoch 87/100
35/35 [==============================] - 55s 2s/step - loss: 6.3003e-04 - val_loss: 0.0166
Epoch 88/100
35/35 [==============================] - 55s 2s/step - loss: 8.6411e-04 - val_loss: 0.0159
Epoch 89/100
35/35 [==============================] - 54s 2s/step - loss: 8.1933e-04 - val_loss: 0.0177
Epoch 90/100
35/35 [==============================] - 55s 2s/step - loss: 0.0

In [94]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=200,
                   validation_data=(x_test, y_test), callbacks=[tfb], initial_epoch=101)

Epoch 102/200
35/35 [==============================] - 54s 2s/step - loss: 3.1096e-04 - val_loss: 0.0149
Epoch 103/200
35/35 [==============================] - 54s 2s/step - loss: 3.0200e-04 - val_loss: 0.0158
Epoch 104/200
35/35 [==============================] - 55s 2s/step - loss: 5.1295e-04 - val_loss: 0.0151
Epoch 105/200
35/35 [==============================] - 53s 2s/step - loss: 4.1727e-04 - val_loss: 0.0148
Epoch 106/200
35/35 [==============================] - 54s 2s/step - loss: 3.9749e-04 - val_loss: 0.0154
Epoch 107/200
35/35 [==============================] - 54s 2s/step - loss: 3.5823e-04 - val_loss: 0.0157
Epoch 108/200
35/35 [==============================] - 54s 2s/step - loss: 3.3754e-04 - val_loss: 0.0155
Epoch 109/200
35/35 [==============================] - 54s 2s/step - loss: 3.1826e-04 - val_loss: 0.0151
Epoch 110/200
35/35 [==============================] - 55s 2s/step - loss: 4.5103e-04 - val_loss: 0.0151
Epoch 111/200
35/35 [==============================] - 

35/35 [==============================] - 88s 3s/step - loss: 4.2564e-04 - val_loss: 0.0154
Epoch 181/200
35/35 [==============================] - 88s 3s/step - loss: 4.7906e-04 - val_loss: 0.0153
Epoch 182/200
35/35 [==============================] - 89s 3s/step - loss: 4.6281e-04 - val_loss: 0.0152
Epoch 183/200
35/35 [==============================] - 90s 3s/step - loss: 4.8400e-04 - val_loss: 0.0156
Epoch 184/200
35/35 [==============================] - 82s 2s/step - loss: 5.1508e-04 - val_loss: 0.0154
Epoch 185/200
35/35 [==============================] - 76s 2s/step - loss: 5.3316e-04 - val_loss: 0.0153
Epoch 186/200
35/35 [==============================] - 80s 2s/step - loss: 5.1010e-04 - val_loss: 0.0155
Epoch 187/200
35/35 [==============================] - 74s 2s/step - loss: 4.7929e-04 - val_loss: 0.0154
Epoch 188/200
35/35 [==============================] - 68s 2s/step - loss: 6.8259e-04 - val_loss: 0.0153
Epoch 189/200
35/35 [==============================] - 66s 2s/step - 

In [95]:
model.save('./models/object_detection.h5')